通过`type()`函数创建的类和直接写class是完全一样的，因为Python解释器遇到`class`定义时，仅仅是扫描一下`class`定义的语法，然后调用`type()`函数创建出`class`。

当我们定义了类以后，就可以根据这个类创建出实例，所以：先定义类，然后创建实例。但是如果我们想创建出类呢？那就必须根据`metaclass`创建出类，所以：先定义`metaclass`，然后创建类。`metaclass`允许你创建类或者修改类。换句话说，你可以把类看成是`metaclass`创建出来的“实例”。

道 即是`type`

一 即是 `metaclass`(元类，或者叫类生成器)

二 即是 `class`(类，或者叫实例生成器)

三 即是 `instance`(实例)

万物 即是 实例的各种属性与方法，我们平常使用python时，调用的就是它们。

In [ ]:
'''元类是由“type”衍生而出，所以父类需要传入type。
元类的操作都在 __new__中完成，它的第一个参数是将创建的类.
之后的参数即是三大永恒命题：我是谁，我从哪里来，我将到哪里去。'''
class SayMetaClass(type):
    def __new__(cls, name, bases, attrs):
        attrs['say_'+name] = lambda self,value,saying=name: print(saying+','+value+'!')
        return type.__new__(cls, name, bases, attrs)
class Hello(object, metaclass=SayMetaClass):
    pass
 
# 二生三：创建实列
hello = Hello()
 # 三生万物：调用实例方法
hello.say_Hello('world!')
#这个metaclass可以给我们自定义的MyList增加一个add方法：
# metaclass是类的模板，所以必须从`type`类型派生：
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)
class MyList(list, metaclass=ListMetaclass):
    pass

当我们传入关键字参数`metaclass`时，魔术就生效了，它指示Python解释器在创建MyList时，要通过`ListMetaclass.__new__()`来创建，在此，我们可以修改类的定义，然后，返回修改后的定义。

`__new__()`方法接收到的参数依次是：当前准备创建的类的对象；类的名字；类继承的父类集合；类的方法集合。

ORM全称“Object Relational Mapping”，即对象-关系映射，就是把关系数据库的一行映射为一个对象，也就是一个类对应一个表，这样，写代码更简单，不用直接操作SQL语句。

In [1]:
'''在Field类实例化时将得到两个参数，name和column_type，
它们将被绑定为Field的私有属性，如果要将Field转化为字符串时，
将返回“Field:XXX” ， XXX是传入的name名称。'''
class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)
    
class StringField(Field):

    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')

class IntegerField(Field):

    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')

In [9]:
'''创建一个新的字典mapping
将每一个类的属性，通过.items()遍历其键值对。如果值是Field类，则打印键值，并将这一对键值绑定到mapping字典上。
将刚刚传入值为Field类的属性删除。
创建一个专门的__mappings__属性，保存字典mapping。
创建一个专门的__table__属性，保存传入的类的名称。'''
class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)
    
class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))

当用户定义一个`class User(Model)`时，Python解释器首先在当前类`User`的定义中查找`metaclass`，如果没有找到，就继续在父类Model中查找`metaclass`，找到了，就使用Model中定义的`metaclass`的`ModelMetaclass`来创建User类，也就是说，`metaclass`可以隐式地继承到子类，但子类自己却感觉不到。

在ModelMetaclass中，一共做了几件事情：排除掉对Model类的修改；在当前类（比如User）中查找定义的类的所有属性，如果找到一个Field属性，就把它保存到一个`__mappings__`的`dict`中，同时从类属性中删除该Field属性，否则，容易造成运行时错误（实例的属性会遮盖类的同名属性）；把表名保存到`__table__`中，这里简化为表名默认为类名。

在Model类中，就可以定义各种操作数据库的方法，比如save()，delete()，find()，update等等。我们实现了save()方法，把一个实例保存到数据库中。因为有表名，属性到字段的映射和属性值的集合，就可以构造出INSERT语句。

In [10]:
'''Model.__setattr__(self, ‘id’, IntegerField(‘id’))
因为IntergerField(‘id’)是Field的子类的实例，自动触发元类的__new__，
所以将IntergerField(‘id’)存入__mappings__并删除这个键值对。'''
class User(Model):
    # 定义类的属性到列的映射：
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')

# 创建一个实例：
u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
# 保存到数据库：
u.save()

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>
SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [12345, 'Michael', 'test@orm.org', 'my-pwd']


###### 序列化


In [16]:
import json

class Student(object):
    def __init__(self, name, age, score):
        self.name = name
        self.age = age
        self.score = score

s = Student('Bob', 20, 88)
# 默认情况下，dumps()方法不知道如何将Student实例变为一个JSON的{}对象。
print(json.dumps(s))
'''可选参数default就是把任意一个对象变成一个可序列为JSON的对象，
我们只需要为Student专门写一个转换函数，再把函数传进去即可
'''
def student2dict(std):
    return {
        'name': std.name,
        'age': std.age,
        'score': std.score
    }

print(json.dumps(s, default=student2dict))
'''通常class的实例都有一个__dict__属性，它就是一个dict，
用来存储实例变量。也有少数例外，比如定义了__slots__的class。'''
print(json.dumps(s, default=lambda obj: obj.__dict__))

ValueError: between 0 and 100

In [1]:
class MetaClass(type):
    def __new__(cls, classname, superclasses, attribute):
        print('in MetaClass',cls,classname,superclasses,
             attribute)
        return type.__new__(cls, classname, superclasses,
                           attribute)
    def __init__(cls, classname, superclasses, attribute):
        print('__init__', cls, classname, superclasses,
             attribute)
        return type.__init__(cls,classname, superclasses,
                            attribute)
class A(metaclass=MetaClass):
    pass

in MetaClass <class '__main__.MetaClass'> A () {'__module__': '__main__', '__qualname__': 'A'}
__init__ <class '__main__.A'> A () {'__module__': '__main__', '__qualname__': 'A'}
